In [ ]:
from sympy import *
import numpy as np




In [ ]:
        Lin_gas = np.matmul((np.transpose(LinearJacobian(theta_0))), np.transpose(error))


In [ ]:
from sympy import *
I = eye(3)
R=Matrix([[1],[2],[3]])
R1 = Matrix([[11],[22],[33]])
T5 = Matrix([[111],[222],[333]])
B=Matrix([[R,R,T5],[R1,R,T5]])
L = np.array(B).astype(np.float64)

[1,2,3,4,5,6][:3]


In [36]:
display(L)
F2 = L[:,-1][:-1]
F2

array([[  1.,   1., 111.],
       [  2.,   2., 222.],
       [  3.,   3., 333.],
       [ 11.,   1., 111.],
       [ 22.,   2., 222.],
       [ 33.,   3., 333.]])

array([111., 222., 333., 111., 222.])

In [ ]:
M=Matrix([I,I])

M


Jacoy = Matrix([[1, -1, 1, 1, 1, 1],
                    [1, -1, 1, 1, 1, 1],
                    [1, -1, 1, 1, 1, 1],
                    [1, -1, 1, 1, 1, 1],
                    [1, -1, 1, 1, 1, 1],
                    [1, -1, 1, 1, 1, 1]])

In [48]:
display(L)
Lin_gas = np.matmul(L, np.array([1,0,0]))
Lin_gas

la = Matrix([[1, -1, 1, 1, 1, 1],
                    [1, -1, 1, 1, 1, 3],
                    [1, -1, 1, 1, 1, 4],
                    [1, -1, 1, 1, 1, 51],
                    [1, -1, 1, 1, 88, 61],
                    [1, -1, 1, 1, 8, 7]])
la2 = la.col(-1)
la2.row_del(-1)
display(la2)

R01 = la
R01.row_del(-1)
R01.col_del(-1)
display(R01)

array([[  1.,   1., 111.],
       [  2.,   2., 222.],
       [  3.,   3., 333.],
       [ 11.,   1., 111.],
       [ 22.,   2., 222.],
       [ 33.,   3., 333.]])

Matrix([
[ 1],
[ 3],
[ 4],
[51],
[61]])

Matrix([
[1, -1, 1, 1,  1],
[1, -1, 1, 1,  1],
[1, -1, 1, 1,  1],
[1, -1, 1, 1,  1],
[1, -1, 1, 1, 88]])

In [52]:
import numpy as np
from sympy import *
from scipy.spatial.transform import Rotation
import pandas as pd
import statistics


alpha1, alpha2, alpha3, alpha4, alpha5, alpha6 = symbols('alpha1:7')
a1, a2, a3, a4, a5, a6 = symbols('a1:7')
d1, d2, d3, d4, d5, d6 = symbols('d1:7')
q1, q2, q3, q4, q5, q6 = symbols('q1:7')
theta_1,theta_2,theta_3,theta_4,theta_5,theta_6=symbols('theta1:7')

theta_list=[theta_1,theta_2,theta_3,theta_4,theta_5,theta_6]
def set_dh_table():
    """

    Returns: dictionary

    Important: all length arguments in [m]
               all angles argument in [radians]

    """

    dh_subs_dict = {alpha1: pi / 2, a1: 0, d1: (128.5 + 115) / 1000, q1: q1,
                    alpha2: pi, a2: 0.28, d2: 0.03, q2: q2 + pi / 2,
                    alpha3: pi / 2, a3: 0, d3: 0.02, q3: q3 + pi / 2,
                    alpha4: pi / 2, a4: 0, d4: 0.245, q4: q4 + pi / 2,
                    alpha5: pi / 2, a5: 0, d5: 0.057, q5: q5 + pi,
                    alpha6: 0, a6: 0, d6: 0.235, q6: q6 + pi / 2}

    return dh_subs_dict


def dh(alpha, a, d, theta):
    """
    Args:
        alpha: torsion angle
        a: distance
        d: translation
        theta: rotation angle

    Returns: Homogeneous DH matrix

    Important note: use sympy cos/sin arguments instead of math/numpy versions.
    i.e: cos(theta) \ sin(theta)

    """
    DH = Matrix([[cos(theta), -sin(theta) * cos(alpha), sin(theta) * sin(alpha), a * cos(theta)],
                 [sin(theta), cos(theta) * cos(alpha), -cos(theta) * sin(alpha), a * sin(theta)],
                 [0, sin(alpha), cos(alpha), d], [0, 0, 0, 1]]
                )
    return DH


def FK(theta_list):
    """
    Args:
        theta_list: joint angle vector ---> list [1,6]
    Returns:
        End effector homogeneous matrix --> Matrix((4, 4))

    Hints:
        - we have added a sympy implementation with missing parts, u dont have to use the same method.
        - chain 'Tes' to T_06 at the end.
    """
    table = set_dh_table()

    T_01 = dh(table[alpha1], table[a1], table[d1], table[q1])
    T_12 = dh(table[alpha2], table[a2], table[d2], table[q2])
    T_23 = dh(table[alpha3], table[a3], table[d3], table[q3])
    T_34 = dh(table[alpha4], table[a4], table[d4], table[q4])
    T_45 = dh(table[alpha5], table[a5], table[d5], table[q5])
    T_56 = dh(table[alpha6], table[a6], table[d6], table[q6])
    Tes = Matrix([[0, -1, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])  # mandatory
    T = T_01 * T_12 * T_23 * T_34 * T_45 * T_56 * Tes

    ''' fill angles to dict for sympy calculations'''

    theta_dict = {}
    keys = [q1, q2, q3, q4, q5, q6]
    for i in range(len(theta_list)):
        theta_dict[keys[i]] = theta_list[i]
    ''' 
    homogeneous transformation matrix from base_link to end_effector [type: numeric matrix] 
    because we are using sympy, we have to use evalf.
    '''
    T_0G_eval = T.evalf(subs=theta_dict, chop=True, maxn=4)

    return T_0G_eval


def xyz_eulerm(A):
    """
    Extract translation and orientation in euler angles

    Args:
        A: Homogeneous transformation --> np.array((4, 4))

    Returns: x, y, z, thetax, thetay, thetaz --> np.array((1, 6))

    Important note: use numpy arrays

    """
    M = np.copy(A)
    M = M[:, -1]
    M = M[:-1]

    R = Rotation.from_matrix(A[:3, :3])
    M2 = R.as_euler('xyz', degrees=True)

    yaw = [np.array(M2[2]).astype(np.float64)]
    roll = [np.array(M2[0]).astype(np.float64)]
    pitch = [np.array(M2[1]).astype(np.float64)]

    arrayte = np.append(M, roll)
    arrayte = np.append(arrayte, pitch)
    arrayte = np.append(arrayte, yaw)

    return np.array([M, roll, pitch, yaw])

def xyz_euler(A):

    """
    Extract translation and orientation in euler angles

    Args:
        A: Homogeneous transformation --> np.array((4, 4))

    Returns: x, y, z, roll, pitch, yaw --> np.array((1, 6))
             x, y, z --> [m]
             roll, pitch, yaw -- > [degrees]

    Important note: use numpy arrays

    """
    x = [np.array(A[0, -1]).astype(np.float64)]
    y = [np.array(A[1, -1]).astype(np.float64)]
    z = [np.array(A[2, -1]).astype(np.float64)]
    R = Rotation.from_matrix(A[:3, :3])
    # Euler = R.as_rotvec(degrees=True)
    Euler = R.as_euler('xyz', degrees=True)

    roll = [np.array(Euler[0]).astype(np.float64)]
    pitch = [np.array(Euler[1]).astype(np.float64)]
    yaw = [np.array(Euler[2]).astype(np.float64)]
    return np.array([x, y, z, roll, pitch, yaw])

def angles_to_follow():
    """

    Returns: Dictionary of the desired angels

    """


    angles = { 't1': [6.244543718,0.205006374 ,2.528057062 , 4.747313019, 5.754333091, 4.771695268],
               't2': [    0.21685716,    5.188549707 ,    5.785487218,    5.477244618,    4.939857742,    4.816323337 ],
                't3': [1.791510664,5.272534951 , 6.060097322,4.41018522 ,4.872383313 , 5.451832625],
                't4': [0.803549588,5.606713142 ,1.354619846 ,5.300093699 , 4.803809327, 0.213959913],
                't5': [0.103271132, 5.470490194, 1.843957808, 4.959719589, 4.499912597,5.949460901 ],
               't6': [0.546672028, 4.717590062, 5.494802631,  4.531869576, 5.41809541,  6.172339446],
               't7':   [1.059135603, 5.721032208, 6.184190232, 4.027556688, 4.967329224,     0.049078659],
               't8':   [4.913328737,    5.150361903,     0.494504137,     0.213244328,     4.67989095,  6.172601246],
               't9':   [5.511505432,     5.31255535,     1.182600195,     5.955587007,     5.659369726,     6.03951989],
               't10': [5.747753199,    5.283006926,     0.66844365,     5.360726438,  5.554318358,  0.601196114],}  # [radians]
    #
    #
    return angles
#pd.display(xyz_euler(np.array([[0, 1,0,0.41],[1,0,0,0],[0,0,-1,0],[0,0,0,1]])))

#print(FK(theta_list))

#theta_list_1=[6.244543718,0.205006374 ,2.528057062 , 4.747313019, 5.754333091, 4.771695268]
#theta_list_2=[    0.21685716,    5.188549707 ,    5.785487218,    5.477244618,    4.939857742,    4.816323337 ]
#theta_list_3=[1.791510664,5.272534951 , 6.060097322,4.41018522 ,4.872383313 , 5.451832625]
#theta_list_4=[0.803549588,5.606713142 ,1.354619846 ,5.300093699 , 4.803809327, 0.213959913]
#theta_list_5=[0.103271132, 5.470490194, 1.843957808, 4.959719589, 4.499912597,5.949460901 ]
#theta_list_6=[0.546672028, 4.717590062, 5.494802631,  4.531869576, 5.41809541,  6.172339446]
#theta_list_7=[1.059135603, 5.721032208, 6.184190232, 4.027556688, 4.967329224,     0.049078659]
#theta_list_8=[4.913328737,    5.150361903,     0.494504137,     0.213244328,     4.67989095,  6.172601246]
#theta_list_9= [5.511505432,     5.31255535,     1.182600195,     5.955587007,     5.659369726,     6.03951989]
theta_list_10=[5.747753199,    5.283006926,     0.66844365,     5.360726438,  5.554318358,  0.601196114]
Tee=FK(theta_list_10)
print(Tee)

x=[0.161,0.549,0.003,0.166,0.086,0.629,0.373,0.325,0.459,0.569]
second_x=[0.386,0.243,0.054,0.223,0.435,0.424,-0.028,-0.247,-0.089,-0.078]
std_x=[]
for i in x:
    std_x_i=np.std([x[i],second_x[i]])
    std_x.append(std_x_i)
print(std_x)
    
y=[-0.068,0.231,0.597,0.276,0.0,0.268,0.265,-0.434,-0.304,-0.292]
second_y=[-0.108,-0.22,-0.147,-0.368,-0.297,-0.323,-0.449,-0.455,-0.61,-0.376]
std_y=[]
for i in y:
    std_y_i=np.std([y[i],second_y[i]])
    std_y.append(std_y_i)    
print(std_y)

z=[0.118,0.503,0.446,0.140,0.153,0.412,0.702,0.343,0.080,0.195]
second_z=[0.479,0.344,0.178,0.137,0.087,0.46,0.424,0.222,0.127,0.621]
std_z=[]
for i in z:
    std_z_i=np.std([z[i],second_z[i]])
    std_z.append(std_z_i) 
print(std_z)

theta_x=[0.183,1.331,0.925,-0.472,-1.027,1.31,1.324,1.356,2.222,2.219]
second_theta_x=[1.545558866,1.544372042,1.54377863,1.550288708,1.555577056,1.37404536,1.358721369,1.358337397,-0.746582041,-0.768643003]


std_theta_x=[]
for i in theta_x:
    std_theta_x_i=np.std([theta_x[i],second_theta_x[i]])
    std_theta_x.append(std_theta_x_i)
print(std_theta_x)


theta_y=[3.105,2.641,2.745,-2.645,-2.087,1.816,1.816,0.058,0.091,0.095]
second_theta_y=[0.014172074,0.013980087,0.016161749,0.014294247,0.013439035,0.093252942,0.062081361,0.063669611,-2.125741216,-2.122809063]
std_theta_y=[]
for i in theta_y:
    std_theta_y_i=np.std([theta_y[i],second_theta_y[i]])
    std_theta_y.append(std_theta_y_i) 
print(std_theta_y)


theta_z=[1.45,2.432,2.718,0.532,0.94,1.726,1.742,1.78,1.634,1.634]
second_theta_z=[2.279749069,2.279975962,2.280377387,2.257565934,2.232677539,2.122459997,0.540563376,0.172473437,2.735646523,2.873091201]
std_theta_z=[]
for i in theta_z:
    std_theta_z_i=np.std([theta_z[i],second_theta_z[i]])
    std_theta_z.append(std_theta_z_i)
print(std_theta_z)


mean_x=[]
for i in x:
    mean_x_i=statistics.mean([x[i],second_x[i]])
    mean_x.append( mean_x_i)
print(mean_x)

mean_y=[]
for i in y:
    mean_y_i=statistics.mean([y[i],second_y[i]])
    mean_y.append( mean_y_i)
print(mean_y)

mean_z=[]
for i in z:
    mean_z_i=statistics.mean([z[i],second_z[i]])
    mean_z.append( mean_z_i)
print(mean_z_i)

mean_theta_x=[]
for i in x:
    mean_theta_x_i=statistics.mean([theta_x[i],second_theta_x[i]])
    mean_theta_x.append( mean_theta_x_i)
print(mean_theta_x)

mean_theta_y=[]
for i in y:
    mean_theta_y_i=statistics.mean([theta_y[i],second_theta_y[i]])
    mean_theta_y.append( mean_theta_y_i)
print(mean_theta_y)

mean_theta_z=[]
for i in z:
    mean_theta_z_i=statistics.mean([theta_z[i],second_theta_z[i]])
    mean_theta_z.append( mean_theta_z_i)
print(mean_theta_z)

Matrix([[-0.0630577554287000, 0.597915775674978, 0.799074742861570, 0.568973251382420], [0.993535108891191, 0.113344292040597, -0.00640771896901770, -0.292096956852906], [-0.0944018372748339, 0.793504755285566, -0.601197551939735, 0.195276531371777], [0, 0, 0, 1.00000000000000]])


TypeError: list indices must be integers or slices, not float

In [54]:
import numpy as np
from sympy import *
from scipy.spatial.transform import Rotation
import pandas as pd
import statistics


alpha1, alpha2, alpha3, alpha4, alpha5, alpha6 = symbols('alpha1:7')
a1, a2, a3, a4, a5, a6 = symbols('a1:7')
d1, d2, d3, d4, d5, d6 = symbols('d1:7')
q1, q2, q3, q4, q5, q6 = symbols('q1:7')
theta_1,theta_2,theta_3,theta_4,theta_5,theta_6=symbols('theta1:7')

theta_list=[theta_1,theta_2,theta_3,theta_4,theta_5,theta_6]
def set_dh_table():
    """

    Returns: dictionary

    Important: all length arguments in [m]
               all angles argument in [radians]

    """

    dh_subs_dict = {alpha1: pi / 2, a1: 0, d1: (128.5 + 115) / 1000, q1: q1,
                    alpha2: pi, a2: 0.28, d2: 0.03, q2: q2 + pi / 2,
                    alpha3: pi / 2, a3: 0, d3: 0.02, q3: q3 + pi / 2,
                    alpha4: pi / 2, a4: 0, d4: 0.245, q4: q4 + pi / 2,
                    alpha5: pi / 2, a5: 0, d5: 0.057, q5: q5 + pi,
                    alpha6: 0, a6: 0, d6: 0.235, q6: q6 + pi / 2}

    return dh_subs_dict


def dh(alpha, a, d, theta):
    """
    Args:
        alpha: torsion angle
        a: distance
        d: translation
        theta: rotation angle

    Returns: Homogeneous DH matrix

    Important note: use sympy cos/sin arguments instead of math/numpy versions.
    i.e: cos(theta) \ sin(theta)

    """
    DH = Matrix([[cos(theta), -sin(theta) * cos(alpha), sin(theta) * sin(alpha), a * cos(theta)],
                 [sin(theta), cos(theta) * cos(alpha), -cos(theta) * sin(alpha), a * sin(theta)],
                 [0, sin(alpha), cos(alpha), d], [0, 0, 0, 1]]
                )
    return DH


def FK(theta_list):
    """
    Args:
        theta_list: joint angle vector ---> list [1,6]
    Returns:
        End effector homogeneous matrix --> Matrix((4, 4))

    Hints:
        - we have added a sympy implementation with missing parts, u dont have to use the same method.
        - chain 'Tes' to T_06 at the end.
    """
    table = set_dh_table()

    T_01 = dh(table[alpha1], table[a1], table[d1], table[q1])
    T_12 = dh(table[alpha2], table[a2], table[d2], table[q2])
    T_23 = dh(table[alpha3], table[a3], table[d3], table[q3])
    T_34 = dh(table[alpha4], table[a4], table[d4], table[q4])
    T_45 = dh(table[alpha5], table[a5], table[d5], table[q5])
    T_56 = dh(table[alpha6], table[a6], table[d6], table[q6])
    Tes = Matrix([[0, -1, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])  # mandatory
    T = T_01 * T_12 * T_23 * T_34 * T_45 * T_56 * Tes

    ''' fill angles to dict for sympy calculations'''

    theta_dict = {}
    keys = [q1, q2, q3, q4, q5, q6]
    for i in range(len(theta_list)):
        theta_dict[keys[i]] = theta_list[i]
    ''' 
    homogeneous transformation matrix from base_link to end_effector [type: numeric matrix] 
    because we are using sympy, we have to use evalf.
    '''
    T_0G_eval = T.evalf(subs=theta_dict, chop=True, maxn=4)

    return T_0G_eval

display("DOR",FK([0.104719755,5.323254219,1.357866158,4.810127418,5.471607205,6.213372137]))

'DOR'

Matrix([
[ -0.994157910274789,  -0.10770225431903,  0.007090405676995,    0.411767865984323],
[ -0.106919057881054,  0.991663216235933, 0.0719192646405152, -0.00709107370862582],
[-0.0147771614288168, 0.0707410063485354, -0.997385254312948, -0.00082842273606332],
[                  0,                  0,                  0,                  1.0]])

In [57]:
import numpy as np
#### Question 5
Tee_star_wanted = np.array([[0, 1, 0,0.41],[1, 0, 0,0],[0, 0, -1,0],[0,0,0,1]])
Tee_reality = np.array([[-0.994, -0.107, 0.007,0.411],[-0.107,0.9991, 0.072,-0.007],[-0.014, 0.070, -0.997,0],[0,0,0,1]])
def compare_2_rot_mat(mat1,mat2):
    """dot producut of corrsesponding coloumns of the matrixes.
    In - 1. array 2D 3x3, 2. array 2D 3x3
    Out - 1D arry, each element is dot product of each columns
    """
    res0=np.dot(mat1[:,0],mat2[:,0])
    res1=np.dot(mat1[:,1],mat2[:,1])
    res2=np.dot(mat1[:,2],mat2[:,2])
    res3=np.dot(mat1[:,3],mat2[:,3])
    return np.array([res0,res1,res2,res3])

print(compare_2_rot_mat(Tee_star_wanted,Tee_reality))

[-0.107   -0.107    0.997    1.16851]


In [1]:
import numpy as np
from sympy import *
from scipy.spatial.transform import Rotation
import pandas as pd
import statistics


alpha1, alpha2, alpha3, alpha4, alpha5, alpha6 = symbols('alpha1:7')
a1, a2, a3, a4, a5, a6 = symbols('a1:7')
d1, d2, d3, d4, d5, d6 = symbols('d1:7')
q1, q2, q3, q4, q5, q6 = symbols('q1:7')
theta_1,theta_2,theta_3,theta_4,theta_5,theta_6=symbols('theta1:7')

theta_list=[theta_1,theta_2,theta_3,theta_4,theta_5,theta_6]
def set_dh_table():
    """

    Returns: dictionary

    Important: all length arguments in [m]
               all angles argument in [radians]

    """

    dh_subs_dict = {alpha1: pi / 2, a1: 0, d1: (128.5 + 115) / 1000, q1: q1,
                    alpha2: pi, a2: 0.28, d2: 0.03, q2: q2 + pi / 2,
                    alpha3: pi / 2, a3: 0, d3: 0.02, q3: q3 + pi / 2,
                    alpha4: pi / 2, a4: 0, d4: 0.245, q4: q4 + pi / 2,
                    alpha5: pi / 2, a5: 0, d5: 0.057, q5: q5 + pi,
                    alpha6: 0, a6: 0, d6: 0.235, q6: q6 + pi / 2}

    return dh_subs_dict


def dh(alpha, a, d, theta):
    """
    Args:
        alpha: torsion angle
        a: distance
        d: translation
        theta: rotation angle

    Returns: Homogeneous DH matrix

    Important note: use sympy cos/sin arguments instead of math/numpy versions.
    i.e: cos(theta) \ sin(theta)

    """
    DH = Matrix([[cos(theta), -sin(theta) * cos(alpha), sin(theta) * sin(alpha), a * cos(theta)],
                 [sin(theta), cos(theta) * cos(alpha), -cos(theta) * sin(alpha), a * sin(theta)],
                 [0, sin(alpha), cos(alpha), d], [0, 0, 0, 1]]
                )
    return DH


def FK(theta_list):
    """
    Args:
        theta_list: joint angle vector ---> list [1,6]
    Returns:
        End effector homogeneous matrix --> Matrix((4, 4))

    Hints:
        - we have added a sympy implementation with missing parts, u dont have to use the same method.
        - chain 'Tes' to T_06 at the end.
    """
    table = set_dh_table()

    T_01 = dh(table[alpha1], table[a1], table[d1], table[q1])
    T_12 = dh(table[alpha2], table[a2], table[d2], table[q2])
    T_23 = dh(table[alpha3], table[a3], table[d3], table[q3])
    T_34 = dh(table[alpha4], table[a4], table[d4], table[q4])
    T_45 = dh(table[alpha5], table[a5], table[d5], table[q5])
    T_56 = dh(table[alpha6], table[a6], table[d6], table[q6])
    Tes = Matrix([[0, -1, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])  # mandatory
    T = T_01 * T_12 * T_23 * T_34 * T_45 * T_56 * Tes

    ''' fill angles to dict for sympy calculations'''

    theta_dict = {}
    keys = [q1, q2, q3, q4, q5, q6]
    for i in range(len(theta_list)):
        theta_dict[keys[i]] = theta_list[i]
    ''' 
    homogeneous transformation matrix from base_link to end_effector [type: numeric matrix] 
    because we are using sympy, we have to use evalf.
    '''
    T_0G_eval = T.evalf(subs=theta_dict, chop=True, maxn=4)

    return T_0G_eval

print("DOR",FK([0.10472,5.32325,1.35787,4.81013,5.47161,6.21337]))

def xyz_eulerm(A):
    """
    Extract translation and orientation in euler angles

    Args:
        A: Homogeneous transformation --> np.array((4, 4))

    Returns: x, y, z, thetax, thetay, thetaz --> np.array((1, 6))

    Important note: use numpy arrays

    """
    M = np.copy(A)
    M = M[:, -1]
    M = M[:-1]

    R = Rotation.from_matrix(A[:3, :3])
    M2 = R.as_euler('xyz', degrees=True)

    yaw = [np.array(M2[2]).astype(np.float64)]
    roll = [np.array(M2[0]).astype(np.float64)]
    pitch = [np.array(M2[1]).astype(np.float64)]

    arrayte = np.append(M, roll)
    arrayte = np.append(arrayte, pitch)
    arrayte = np.append(arrayte, yaw)

    return np.array([M, roll, pitch, yaw])

def xyz_euler(A):

    """
    Extract translation and orientation in euler angles

    Args:
        A: Homogeneous transformation --> np.array((4, 4))

    Returns: x, y, z, roll, pitch, yaw --> np.array((1, 6))
             x, y, z --> [m]
             roll, pitch, yaw -- > [degrees]

    Important note: use numpy arrays

    """
    x = [np.array(A[0, -1]).astype(np.float64)]
    y = [np.array(A[1, -1]).astype(np.float64)]
    z = [np.array(A[2, -1]).astype(np.float64)]
    R = Rotation.from_matrix(A[:3, :3])
    # Euler = R.as_rotvec(degrees=True)
    Euler = R.as_euler('xyz', degrees=True)

    roll = [np.array(Euler[0]).astype(np.float64)]
    pitch = [np.array(Euler[1]).astype(np.float64)]
    yaw = [np.array(Euler[2]).astype(np.float64)]
    return np.array([x, y, z, roll, pitch, yaw])

def angles_to_follow():
    """

    Returns: Dictionary of the desired angels

    """

    angles = { 't1': [6.244543718,0.205006374 ,2.528057062 , 4.747313019, 5.754333091, 4.771695268],
               't2': [    0.21685716,    5.188549707 ,    5.785487218,    5.477244618,    4.939857742,    4.816323337 ],
                't3': [1.791510664,5.272534951 , 6.060097322,4.41018522 ,4.872383313 , 5.451832625],
                't4': [0.803549588,5.606713142 ,1.354619846 ,5.300093699 , 4.803809327, 0.213959913],
                't5': [0.103271132, 5.470490194, 1.843957808, 4.959719589, 4.499912597,5.949460901 ],
               't6': [0.546672028, 4.717590062, 5.494802631,  4.531869576, 5.41809541,  6.172339446],
               't7':   [1.059135603, 5.721032208, 6.184190232, 4.027556688, 4.967329224,     0.049078659],
               't8':   [4.913328737,    5.150361903,     0.494504137,     0.213244328,     4.67989095,  6.172601246],
               't9':   [5.511505432,     5.31255535,     1.182600195,     5.955587007,     5.659369726,     6.03951989],
               't10': [5.747753199,    5.283006926,     0.66844365,     5.360726438,  5.554318358,  0.601196114],}  # [radians]
    #
    #
    return angles




#print(FK(theta_list))

#theta_list_1=[6.244543718,0.205006374 ,2.528057062 , 4.747313019, 5.754333091, 4.771695268]
#theta_list_2=[    0.21685716,    5.188549707 ,    5.785487218,    5.477244618,    4.939857742,    4.816323337 ]
#theta_list_3=[1.791510664,5.272534951 , 6.060097322,4.41018522 ,4.872383313 , 5.451832625]
#theta_list_4=[0.803549588,5.606713142 ,1.354619846 ,5.300093699 , 4.803809327, 0.213959913]
#theta_list_5=[0.103271132, 5.470490194, 1.843957808, 4.959719589, 4.499912597,5.949460901 ]
#theta_list_6=[0.546672028, 4.717590062, 5.494802631,  4.531869576, 5.41809541,  6.172339446]
#theta_list_7=[1.059135603, 5.721032208, 6.184190232, 4.027556688, 4.967329224,     0.049078659]
#theta_list_8=[4.913328737,    5.150361903,     0.494504137,     0.213244328,     4.67989095,  6.172601246]
#theta_list_9= [5.511505432,     5.31255535,     1.182600195,     5.955587007,     5.659369726,     6.03951989]


# theta_list_10=[5.747753199,    5.283006926,     0.66844365,     5.360726438,  5.554318358,  0.601196114]
# Tee=FK(theta_list_10)
# print(Tee)






####################
#####EDEN
####################


x=[0.386,0.243,0.054,0.223,0.435,0.424,-0.028,-0.247,-0.089,-0.078]
x2=[0.39,0.24,0.05,0.22,0.44,0.42,-0.03,-0.25,-0.09,-0.08]

y=[-0.108,-0.22,-0.147,-0.368,-0.297,-0.323,-0.449,-0.455,-0.61,-0.376]
y2=[-0.11,-0.22,-0.15,-0.37,-0.3,-0.32,-0.45,-0.45,-0.61,-0.38]

z=[0.479,0.344,0.178,0.137,0.087,0.46,0.424,0.222,0.127,0.621]
z2=[0.48,0.34,0.18,0.14,0.09,0.46,0.42,0.22,0.13,0.62]

theta_x = [88.5, 88.5, 88.5, 88.8, 89.1, 78.7, 77.85, 77.8, -42.7, -44]
theta_x2 = [88.45, 88.55, 88.52, 88.82, 89.14, 78.6, 77.78, 77.6, -42.5, -43.92]

theta_y = [0.81, 0.8, 0.92, 0.82, 0.77, 5.3, 3.5, 3.65, -121.8, -121.6]
theta_y2 = [0.796, 0.792, 0.91, 0.82, 0.78, 5.35, 3.42, 3.64, -121.4, -121.56]
theta_z = [130.6, 130.6, 130.6, 129.3, 127.9, 121.6, 30.9, 9.39, 156.74, 164.6]
theta_z2 = [130.442, 130.255, 130.65, 129.2, 127.6, 121.4, 31.2, 9.83, 156.72, 164.55]




print('mort\n\n')
std_x=[]
for i in range(len(x)):
    std_x_i=np.std([x[i],x2[i]])
    std_x.append(std_x_i)
print('std_x', std_x)

print('mort\n')

std_y=[]
for i in range(len(y)):
    std_y_i=np.std([y[i],y2[i]])
    std_y.append(std_y_i)    
print('std_y', std_y)
print('mort\n')

std_z=[]
for i in range(len(z)):
    std_z_i=np.std([z[i],z2[i]])
    std_z.append(std_z_i) 
print('std_z', std_z)

print('mort\n')


std_theta_x=[]
for i in range(len(theta_x)):
    std_theta_x_i=np.std([theta_x[i],theta_x2[i]])
    std_theta_x.append(std_theta_x_i)
print(std_theta_x)
print('mort\n')


std_theta_y=[]
for i in range(len(theta_y)):
    std_theta_y_i=np.std([theta_y[i],theta_y2[i]])
    std_theta_y.append(std_theta_y_i) 
print(std_theta_y)
print('mort\n')


std_theta_z=[]
for i in range(len(theta_z)):
    std_theta_z_i=np.std([theta_z[i],theta_z2[i]])
    std_theta_z.append(std_theta_z_i)
print(std_theta_z)
print('mort\n')


mean_x=[]
for i in range(len(x)):
    mean_x_i=statistics.mean([x[i],x2[i]])
    mean_x.append( mean_x_i)
print('mean_x',mean_x)
print('mort\n')
mean_y=[]
for i in range(len(y)):
    mean_y_i=statistics.mean([y[i],y2[i]])
    mean_y.append( mean_y_i)
print('mean_y',mean_y)

print('mort\n')
mean_z=[]
for i in range(len(z)):
    mean_z_i=statistics.mean([z[i],z2[i]])
    mean_z.append( mean_z_i)
print('mean_z',mean_z_i)

print('mort\n')
mean_theta_x=[]
for i in range(len(x)):
    mean_theta_x_i=statistics.mean([theta_x[i],theta_x2[i]])
    mean_theta_x.append( mean_theta_x_i)
print('mean_theta_x',mean_theta_x)
print('mort\n')

mean_theta_y=[]
for i in range(len(y)):
    mean_theta_y_i=statistics.mean([theta_y[i],theta_y2[i]])
    mean_theta_y.append( mean_theta_y_i)
print('mean_theta_y',mean_theta_y)

print('mort\n')
mean_theta_z=[]
for i in range(len(z)):
    mean_theta_z_i=statistics.mean([theta_z[i],theta_z2[i]])
    mean_theta_z.append(mean_theta_z_i)
print('mean_theta_z',mean_theta_z)
print('done')

#comparison of matrices
#results=[]
#for A,B in pairs:
   #results=np.zeros((4,4))
   #for i in range(4):
      #result[:,i]=np.dot(A[:,i],A2[:,i])
    #results.append(result)
A=np.array([-0.119,-0.951,0.286,0.185],[-0.992,0.126,0.007,-0.070],[-0.042,-0.283,-0.958,0.124],[0,0,0,1])
A2=np.array([0.106,0.029,-0.745,0.658],[-0.992,-0.1,-0.043,-0.027],[-0.064,-0.292,-0.665,-0.69],[0,0,0,1])
result=np.zeros((4,4))
for i in range(4):
    result[:,i]=np.dot(A[:,i],A2[:,i])
results.append(result)
#for i, result in enumerate(results);
print(result)

DOR Matrix([[-0.994157943100307, -0.107702299575514, 0.00708511374026211, 0.411765834412812], [-0.106919460643720, 0.991663090651940, 0.0719203974842945, -0.00709077601108411], [-0.0147720379847324, 0.0707426978902345, -0.997385210231728, -0.000830906135278978], [0, 0, 0, 1.00000000000000]])
mort


std_x [0.0020000000000000018, 0.0015000000000000013, 0.0019999999999999983, 0.0015000000000000013, 0.0025000000000000022, 0.0020000000000000018, 0.0009999999999999992, 0.0015000000000000013, 0.0005000000000000004, 0.0010000000000000009]
mort

std_y [0.0010000000000000009, 0.0, 0.0015000000000000013, 0.0010000000000000009, 0.0015000000000000013, 0.0015000000000000013, 0.0005000000000000004, 0.0025000000000000022, 0.0, 0.0020000000000000018]
mort

std_z [0.0005000000000000004, 0.001999999999999974, 0.0010000000000000009, 0.0015000000000000013, 0.0015000000000000013, 0.0, 0.0020000000000000018, 0.0010000000000000009, 0.0015000000000000013, 0.0005000000000000004]
mort

[0.02499999999999858, 0.02

TypeError: array() takes from 1 to 2 positional arguments but 4 were given